# Week 3 - Welcome to IBM Data Science Capstone Project
# Start of Section 1

## Import Section

In [2]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

In [80]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported successfully.')

Libraries imported successfully.


# Start of Section 3

In [81]:
#Reading saved csv file to continue progress from Section 2

df_merged = pd.read_csv('Merged_Data.csv').iloc[:, 1:]
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [82]:
#Will be used later to merge dataframes
df_later = df_merged.loc[:, ['Borough', 'Neighborhood']]
df_later.head()

,Borough,Neighborhood
0,Scarborough,"Rouge, Malvern"
1,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,Scarborough,"Guildwood, Morningside, West Hill"
3,Scarborough,Woburn
4,Scarborough,Cedarbrae


## Finding Coordinates and preparing Folium

In [83]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [84]:
#Creating Toronto map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Add markers to Toronto map
for lat, lng, postalcode, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'],df_merged['Postal Code'], df_merged['Borough'], df_merged['Neighborhood']):
    label = '{}, {}, {}'.format(postalcode, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#e334c3',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Foursquare Section
### Exploring Various Places in Neighborhoods

In [85]:
#Initialising Foursquare Credentials
CLIENT_ID = 'BAMVZJNDJFCXQ3KDKHU1TFVEH3ERVHOSW4X1EL4H50BUWKRN' # your Foursquare ID
CLIENT_SECRET = 'GZ0W1BD4KNYN4FNURB2QB5BT0YWUB4ZNOYYB5GH5MQGG5XTW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
limit = 5

Your credentials:
CLIENT_ID: BAMVZJNDJFCXQ3KDKHU1TFVEH3ERVHOSW4X1EL4H50BUWKRN
CLIENT_SECRET:GZ0W1BD4KNYN4FNURB2QB5BT0YWUB4ZNOYYB5GH5MQGG5XTW


In [86]:
#Defining Function to retrieve venues in the vicinity
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
     
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
toronto_venues = getNearbyVenues(names=df_merged['Neighborhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

In [88]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[99]] + list(toronto_onehot.columns[:98]) + list(toronto_onehot.columns[100:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Noodle House,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [89]:
#Code for grouping by Neighborhood (not used)
#toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#num_top_venues = 5

#for hood in toronto_grouped['Neighborhood']:
#    print("----"+hood+"----")
#    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
#    temp.columns = ['venue','freq']
#    temp = temp.iloc[1:]
#    temp['freq'] = temp['freq'].astype(float)
#    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#    print('\n')
        
#Grouping the one hot dataframe by Borough
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_merged = df_later.merge(toronto_grouped, on='Neighborhood')
toronto_merged_drop = toronto_merged.drop('Neighborhood', axis=1)
toronto_grouped_borough = toronto_merged_drop.groupby('Borough').mean().reset_index()
toronto_grouped_borough.head()

,Borough,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Noodle House,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.044444,0.000000,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.00,0.0,0.022222,0.00,0.0,0.000000,0.00,0.00,0.037037,0.0,0.00,0.0,0.044444,0.0,0.022222,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.022222,0.044444,0.022222,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.022222,0.0,0.00,0.0,0.037037,0.00,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,0.022222,0.037037,0.044444,0.0,0.000000,0.000000,0.027778,0.0,0.0,0.022222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022222,0.022222,0.000,0.0,0.0,0.0,0.0,0.000000,0.087037,0.00,0.00,0.022222,0.055556,0.000000,0.037037,0.000000,0.000000,0.000000,0.00,0.022222,0.000000,0.000000,0.022222,0.000,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.022222,0.00,0.05,0.037037,0.000000,0.055556,0.000000,0.000000,0.0,0.027778,0.022222,0.0,0.00,0.00,0.0,0.022222
1,Downtown Toronto,0.010526,0.010526,0.010526,0.010526,0.010526,0.010526,0.0,0.0,0.021053,0.00,0.0,0.00,0.0,0.010526,0.00,0.00,0.0,0.010526,0.00,0.0,0.010526,0.00,0.00,0.000000,0.0,0.00,0.0,0.042105,0.0,0.000000,0.010526,0.115789,0.0,0.010526,0.021053,0.0,0.000000,0.00,0.0,0.00,0.010526,0.010526,0.000000,0.010526,0.021053,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.00,0.010526,0.052632,0.010526,0.0,0.010526,0.00,0.0,0.0,0.021053,0.000000,0.010526,0.0,0.052632,0.031579,0.000000,0.0,0.0,0.010526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021053,0.000000,0.000,0.0,0.0,0.0,0.0,0.010526,0.068421,0.00,0.00,0.010526,0.013158,0.021053,0.000000,0.010526,0.021053,0.010526,0.00,0.094737,0.000000,0.010526,0.000000,0.000,0.0,0.00,0.0,0.0,0.010526,0.0,0.010526,0.00,0.0,0.000000,0.00,0.00,0.000000,0.010526,0.000000,0.010526,0.010526,0.0,0.013158,0.031579,0.0,0.00,0.00,0.0,0.000000
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.04,0.0,0.000000,0.04,0.0,0.000000,0.00,0.

In [90]:
num_top_venues = 5

for bor in toronto_grouped_borough['Borough']:
    print("----"+bor+"----")
    temp = toronto_grouped_borough[toronto_grouped_borough['Borough'] == bor].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
              venue  freq
0              Park  0.09
1        Playground  0.06
2      Tennis Court  0.06
3  Sushi Restaurant  0.05
4      Dessert Shop  0.04


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.12
1          Restaurant  0.09
2                Park  0.07
3                 Gym  0.05
4  Italian Restaurant  0.05


----East Toronto----
                venue  freq
0      Ice Cream Shop  0.12
1    Greek Restaurant  0.08
2         Pizza Place  0.08
3  Italian Restaurant  0.04
4      Cosmetics Shop  0.04


----East York----
               venue  freq
0           Pharmacy  0.12
1                Gym  0.08
2               Park  0.07
3  Convenience Store  0.07
4        Coffee Shop  0.07


----Etobicoke----
                 venue  freq
0          Pizza Place  0.15
1          Coffee Shop  0.06
2       Sandwich Place  0.06
3     Business Service  0.05
4  Rental Car Location  0.05


----Mississauga----
            venue  freq
0           

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Borough'] = toronto_grouped_borough['Borough']

for ind in np.arange(toronto_grouped_borough.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped_borough.iloc[ind, :], num_top_venues)

toronto_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Park,Tennis Court,Playground,Sushi Restaurant,Indian Restaurant,Café,American Restaurant,Dessert Shop,Bus Line,Pool
1,Downtown Toronto,Coffee Shop,Restaurant,Park,Gym,Italian Restaurant,Café,Japanese Restaurant,Vegetarian / Vegan Restaurant,Pub,Plaza
2,East Toronto,Ice Cream Shop,Greek Restaurant,Pizza Place,Comic Shop,Italian Restaurant,Park,Pet Store,Coffee Shop,Pub,Sandwich Place
3,East York,Pharmacy,Gym,Convenience Store,Park,Coffee Shop,Beer Store,Gym / Fitness Center,Fish & Chips Shop,Indian Restaurant,Liquor Store
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Business Service,Rental Car Location,Baseball Field,Drugstore,Fried Chicken Joint,Beer Store,Liquor Store
5,Mississauga,Hotel,Coffee Shop,Burrito Place,Sandwich Place,Yoga Studio,Diner,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
6,North York,Park,Coffee Shop,Baseball Field,Gym,Grocery Store,Pizza Place,Café,Bank,Discount Store,Japanese Restaurant
7,Scarborough,Playground,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bar,Skating Rink,American Restaurant,Park,Motel,Moving Target
8,West Toronto,Coffee Shop,Bar,Brewery,Italian Restaurant,Movie Theater,Café,Dog Run,Dessert Shop,Eastern European Restaurant,Pub
9,York,Park,Convenience Store,Hockey Arena,Playground,Bus Line,Market,Field,Pizza Place,Grocery Store,Trail


In [93]:
### Code for Sorting by Neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


## Try to Cluster Neighborhoods

In [94]:
#Determine number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [96]:
toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]
label0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]
label01 = label0.groupby(['1st Most Common Venue'])['Borough'].nunique()
label02 = label0.groupby(['2nd Most Common Venue'])['Borough'].nunique()
#label03 = label0.groupby(['3rd Most Common Venue'])['Borough'].nunique()
#label04 = label0.groupby(['4th Most Common Venue'])['Borough'].nunique()
#label05 = label0.groupby(['5th Most Common Venue'])['Borough'].nunique()
#label06 = label0.groupby(['6th Most Common Venue'])['Borough'].nunique()
#label07 = label0.groupby(['7th Most Common Venue'])['Borough'].nunique()
#label08 = label0.groupby(['8th Most Common Venue'])['Borough'].nunique()
#label09 = label0.groupby(['9th Most Common Venue'])['Borough'].nunique()
#label10 = label0.groupby(['10th Most Common Venue'])['Borough'].nunique()
grouped0 = pd.DataFrame([label01, label02])#, label03, label04]) #label05, label06, label07, label08, label09, label10])
grouped0.groupby(grouped0.index).count()

,Airport,Bakery,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Café,Concert Hall,Dessert Shop,Drugstore,Electronics Store,Greek Restaurant,Gym,Hakka Restaurant,Indian Restaurant,Italian Restaurant,Liquor Store,Motel,Movie Theater,Moving Target,Noodle House,Pharmacy,Playground,Pool,Restaurant,Sporting Goods Shop,Yoga Studio,Airport Food Court,American Restaurant,Bank,Bar,Beer Bar,Caribbean Restaurant,Chinese Restaurant,Clothing Store,College Stadium,Cosmetics Shop,Dance Studio,Diner,Field,Fish & Chips Shop,Garden,Golf Course,Intersection,Japanese Restaurant,Lounge,Middle Eastern Restaurant,Pet Store,Pizza Place,Plaza,Rental Car Location,Salon / Barbershop,Shopping Mall,Wings Joint
Borough,1,1,1,1,1,1,2,1,2,1,1,1,1,1,2,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, 9))]]
label0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]
label01 = label0.groupby(['1st Most Common Venue'])['Borough'].nunique()
label02 = label0.groupby(['2nd Most Common Venue'])['Borough'].nunique()
#label03 = label0.groupby(['3rd Most Common Venue'])['Borough'].nunique()
#label04 = label0.groupby(['4th Most Common Venue'])['Borough'].nunique()
#label05 = label0.groupby(['5th Most Common Venue'])['Borough'].nunique()
#label06 = label0.groupby(['6th Most Common Venue'])['Borough'].nunique()
#label07 = label0.groupby(['7th Most Common Venue'])['Borough'].nunique()
#label08 = label0.groupby(['8th Most Common Venue'])['Borough'].nunique()
#label09 = label0.groupby(['9th Most Common Venue'])['Borough'].nunique()
#label10 = label0.groupby(['10th Most Common Venue'])['Borough'].nunique()
grouped0 = pd.DataFrame([label01, label02])#, label03, label04]) #label05, label06, label07, label08, label09, label10])
grouped0.groupby(grouped0.index).count()

,Airport,Bakery,Breakfast Spot,Convenience Store,Fast Food Restaurant,Grocery Store,Gym,Martial Arts Dojo,Park,Playground,River,Speakeasy,Sporting Goods Shop,Thai Restaurant,Bank,Bar,Falafel Restaurant,Food & Drink Shop,Market,Pool,Salad Place,Snack Place,Swim School,Tennis Court,Trail,Yoga Studio
Borough,1,1,1,2,2,1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, 9))]]
label0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]
label01 = label0.groupby(['1st Most Common Venue'])['Borough'].nunique()
label02 = label0.groupby(['2nd Most Common Venue'])['Borough'].nunique()
#label03 = label0.groupby(['3rd Most Common Venue'])['Borough'].nunique()
#label04 = label0.groupby(['4th Most Common Venue'])['Borough'].nunique()
#label05 = label0.groupby(['5th Most Common Venue'])['Borough'].nunique()
#label06 = label0.groupby(['6th Most Common Venue'])['Borough'].nunique()
#label07 = label0.groupby(['7th Most Common Venue'])['Borough'].nunique()
#label08 = label0.groupby(['8th Most Common Venue'])['Borough'].nunique()
#label09 = label0.groupby(['9th Most Common Venue'])['Borough'].nunique()
#label10 = label0.groupby(['10th Most Common Venue'])['Borough'].nunique()
grouped0 = pd.DataFrame([label01, label02])#, label03, label04]) #label05, label06, label07, label08, label09, label10])
grouped0.groupby(grouped0.index).count()

,Brewery,Chinese Restaurant,Comic Shop,Grocery Store,Gym / Fitness Center,Mobile Phone Shop,Pizza Place,Restaurant,Coffee Shop,Cosmetics Shop,Discount Store,Empanada Restaurant,Fried Chicken Joint,Gastropub,Gym,Health Food Store,Park,Sandwich Place,Tea Room
Borough,1,1,1,2,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1


In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, 9))]]
label0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]
label01 = label0.groupby(['1st Most Common Venue'])['Borough'].nunique()
label02 = label0.groupby(['2nd Most Common Venue'])['Borough'].nunique()
#label03 = label0.groupby(['3rd Most Common Venue'])['Borough'].nunique()
#label04 = label0.groupby(['4th Most Common Venue'])['Borough'].nunique()
#label05 = label0.groupby(['5th Most Common Venue'])['Borough'].nunique()
#label06 = label0.groupby(['6th Most Common Venue'])['Borough'].nunique()
#label07 = label0.groupby(['7th Most Common Venue'])['Borough'].nunique()
#label08 = label0.groupby(['8th Most Common Venue'])['Borough'].nunique()
#label09 = label0.groupby(['9th Most Common Venue'])['Borough'].nunique()
#label10 = label0.groupby(['10th Most Common Venue'])['Borough'].nunique()
grouped0 = pd.DataFrame([label01, label02])#, label03, label04]) #label05, label06, label07, label08, label09, label10])
grouped0.groupby(grouped0.index).count()

,Boutique,Breakfast Spot,Bridal Shop,Café,Coffee Shop,Convenience Store,Discount Store,Grocery Store,Gym,Hotel,Indian Restaurant,Massage Studio,Pet Store,Portuguese Restaurant,Restaurant,Bar,Bookstore,Gym / Fitness Center,Hobby Shop,Italian Restaurant,Korean Restaurant,Park,Vietnamese Restaurant
Borough,1,1,1,2,2,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, 9))]]
label0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]
label01 = label0.groupby(['1st Most Common Venue'])['Borough'].nunique()
label02 = label0.groupby(['2nd Most Common Venue'])['Borough'].nunique()
#label03 = label0.groupby(['3rd Most Common Venue'])['Borough'].nunique()
#label04 = label0.groupby(['4th Most Common Venue'])['Borough'].nunique()
#label05 = label0.groupby(['5th Most Common Venue'])['Borough'].nunique()
#label06 = label0.groupby(['6th Most Common Venue'])['Borough'].nunique()
#label07 = label0.groupby(['7th Most Common Venue'])['Borough'].nunique()
#label08 = label0.groupby(['8th Most Common Venue'])['Borough'].nunique()
#label09 = label0.groupby(['9th Most Common Venue'])['Borough'].nunique()
#label10 = label0.groupby(['10th Most Common Venue'])['Borough'].nunique()
grouped0 = pd.DataFrame([label01, label02])#, label03, label04]) #label05, label06, label07, label08, label09, label10])
grouped0.groupby(grouped0.index).count()

,Gym,Dessert Shop
Borough,1,1


In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, 9))]]
label0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]
label01 = label0.groupby(['1st Most Common Venue'])['Borough'].nunique()
label02 = label0.groupby(['2nd Most Common Venue'])['Borough'].nunique()
#label03 = label0.groupby(['3rd Most Common Venue'])['Borough'].nunique()
#label04 = label0.groupby(['4th Most Common Venue'])['Borough'].nunique()
#label05 = label0.groupby(['5th Most Common Venue'])['Borough'].nunique()
#label06 = label0.groupby(['6th Most Common Venue'])['Borough'].nunique()
#label07 = label0.groupby(['7th Most Common Venue'])['Borough'].nunique()
#label08 = label0.groupby(['8th Most Common Venue'])['Borough'].nunique()
#label09 = label0.groupby(['9th Most Common Venue'])['Borough'].nunique()
#label10 = label0.groupby(['10th Most Common Venue'])['Borough'].nunique()
grouped0 = pd.DataFrame([label01, label02])#, label03, label04]) #label05, label06, label07, label08, label09, label10])
grouped0.groupby(grouped0.index).count()

,Baseball Field,Food Truck,Business Service,Yoga Studio
Borough,2,1,1,1


## INTERPRETATION

In [102]:
print("By adding the 1st and 2nd most popular venues from each cluster, we can further describe the clusters as follows: \
\n1st cluster: lots of restaurant and culinary options. \
\n2nd cluster: mixture of restaurant and sport facilities with some playgrounds.\
\n3rd cluster: mix of restaurants, stores, and sport facilities, dominated by pizza place and grocery store. \
\n4th cluster: some shopping venues, relaxation and health facilities, stores, cafes.\
\n5th cluster: gym region with dessert shop. \
\n6th cluster: baseball field region. \
\nThe clustering process is not good enough to distinguish unique features from each cluster \n\
so it is rather difficult to differentiate between clusters.")

By adding the 1st and 2nd most popular venues from each cluster, we can further describe the clusters as follows: 
1st cluster: lots of restaurant and culinary options. 
2nd cluster: mixture of restaurant and sport facilities with some playgrounds.
3rd cluster: mix of restaurants, stores, and sport facilities, dominated by pizza place and grocery store. 
4th cluster: some shopping venues, relaxation and health facilities, stores, cafes.
5th cluster: gym region with dessert shop. 
6th cluster: baseball field region. 
The clustering process is not good enough to distinguish unique features from each cluster 
so it is rather difficult to differentiate between clusters.


# End of Section 3
## Thank you very much.